## Feature Extraction 코드
### 주제 1: 머신러닝 기반 매출의 영향 요인 분석
- feature: 입지, 인구 요인
- target data: 깐부치킨, 생활맥주
- y = 성공 or 실패

In [2]:
## import module
import pandas as pd

## Load Data
# 깐부치킨 매장별 행정동 데이터 Load
store_dong = pd.read_csv("../../resources/processedData/깐부치킨_매장좌표_행정동 추가.csv", encoding="utf8")

# 깐부치킨 거래데이터 Load
sales = pd.read_csv("../../resources/processedData/깐부치킨_거래데이터.csv", encoding="utf8")

# 행정동별 인구 feature Load
people = pd.read_csv("../../resources/processedData/강남구 행정동별 인구 feature.csv", encoding="utf8")

# 매장별 반경 500m 토지 피복 비율 데이터 Load
store_landscape = pd.read_csv("../../resources/processedData/입지특성_매장별_매장반경 500m feature.csv", encoding="utf8")

# 행정동별 토지 면적 데이터
store_landarea = pd.read_csv("../../resources/processedData/입지특성_매장별_매장반경 500m 토지피복_면적.csv", encoding="utf8")

In [ ]:
# 매장별 행정동 추출
store_dong
dong_list = list(store_dong["행정동"].unique())
dong = pd.DataFrame(dong_list, columns=["행정동"])

# 매장 이름 추출
store_list = list(store_dong["매장명"])
store_list

In [ ]:
## store feature 뽑기
store_feature = pd.merge(store_dong[["매장코드","행정동"]], store_landscape.iloc[:,0:5], on= "매장코드", how="right")
store_feature = pd.merge(store_feature, people[["행정동","인구수","유동인구비율"]], on = "행정동", how = "left")
store_feature = pd.merge(store_feature, store_landarea.iloc[:,1:], on = "매장명", how = "left")
store_feature["인구당매출"] = 0

for store in store_list:
    idx = store_feature[store_feature["매장명"] == store].index
    sale = round(sales[sales["매장명"] == store]["총판매금액"].sum() / store_feature[store_feature["매장명"]==store]["인구수"].values[0], 3)
    store_feature.iloc[idx,-1] = sale

In [ ]:
# 데이터 확인
store_feature